# Cleaning

### Motivacion
Dejar los datos en un archivo parquet para poder ser utilizados en el modelo de clasificacion de texto, para eso se eliminan las columnas que no son relevantes y se limpian los textos para que sean mas faciles de procesar.

### Importar librerias

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk

### Importar los datos

In [4]:
def read_data(file_path):
    """
    Lee un archivo CSV y lo carga en un DataFrame de pandas.

    Parámetros
    ----------
    file_path: str
        Ruta al archivo CSV que se desea leer.

    Retorna
    -------
    pandas.DataFrame
        DataFrame con los datos cargados desde el archivo CSV.
    """
    data = pd.read_csv(file_path, encoding="ISO-8859-1")
    return data

In [5]:
def drop_columns(df, columns):
    """
    Elimina una o varias columnas de un DataFrame de pandas.

    Parámetros
    ----------
    df: pandas.DataFrame
        DataFrame del cual se eliminarán las columnas.
    columns: str o list de str
        Nombre o lista de nombres de las columnas a eliminar del DataFrame.

    Retorna
    -------
    pandas.DataFrame
        El mismo DataFrame después de eliminar las columnas indicadas.
    """
    df.drop(columns=columns, inplace=True)
    return df

In [6]:
def merge_datasets(df_left, df_right, merge_column):
    """
    Combina dos DataFrames de pandas mediante un join interno sobre una columna común.

    Parámetros
    ----------
    df_left: pandas.DataFrame
        DataFrame izquierdo en la operación de merge.
    df_right: pandas.DataFrame
        DataFrame derecho en la operación de merge.
    merge_column: str
        Nombre de la columna sobre la cual se realizará la combinación. Debe existir en ambos DataFrames.

    Retorna
    -------
    pandas.DataFrame
        DataFrame resultante de la combinación de df_left y df_right.
    """
    df_merge = df_left.merge(df_right, on=merge_column)
    return df_merge

In [7]:
def group_tags(df, group_column, agg_column):
    """
    Agrupa un DataFrame de pandas por una columna y agrega los valores de otra columna en listas.

    Parámetros
    ----------
    df: pandas.DataFrame
        DataFrame a agrupar.
    group_column: str
        Nombre de la columna por la cual agrupar.
    agg_column: str
        Nombre de la columna cuyos valores se agregarán en listas por cada grupo.

    Retorna
    -------
    pandas.DataFrame
        DataFrame con las columnas `group_column` y `agg_column`, donde esta última contiene listas de valores agrupados.

    """
    df_grouped = df.groupby(group_column)[agg_column].apply(list).reset_index()
    return df_grouped

In [8]:
def get_frecuency_tags(df, column, n=100):
    """
    Extrae las N etiquetas más frecuentes de un DataFrame donde cada fila contiene una lista de etiquetas.

    Parámetros
    ----------
    df: pandas.DataFrame
        DataFrame que contiene la columna de etiquetas.
    column: str
        Nombre de la columna cuyas celdas son listas de etiquetas.
    n: int, opcional
        Número de etiquetas más frecuentes a devolver (por defecto es 100).

    Retorna
    -------
    list of str
        Lista de las N etiquetas más frecuentes, ordenadas de mayor a menor frecuencia.
    """
    flat_list = [item for sublist in df[column].values for item in sublist]
    keywords = nltk.FreqDist(flat_list)
    frequencies_words = keywords.most_common(n)
    tags_features = [word[0] for word in frequencies_words]
    return tags_features

In [9]:
def most_common(tags):
    """
    Filtra una lista de etiquetas conservando solo aquellas presentes en la lista global `tags_features`.

    Parámetros
    ----------
    tags: list of str
        Lista de etiquetas a filtrar.

    Retorna
    -------
    list of str
        Subconjunto de `tags` que también se encuentran en `tags_features`, conservando el orden original.
    """
    tags_filtered = []
    for i in range(len(tags)):
        if tags[i] in tags_features:
            tags_filtered.append(tags[i])
    return tags_filtered

In [10]:
def clean_text(text):
    """
    Limpia y normaliza una cadena de texto en inglés.

    Convierte el texto a minúsculas, expande contracciones comunes
    (p. ej., "what's" → "what is", "can't" → "can not"), elimina secuencias
    de escape específicas y quita espacios al principio y al final.

    Parámetros
    ----------
    text: str
        Cadena de texto que se desea limpiar y normalizar.

    Retorna
    -------
    str
        Texto procesado: en minúsculas, contracciones expandidas y sin espacios
        iniciales o finales.
    """
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = text.strip(' ')
    return text

In [11]:
def data_transformation(data):
    """
    Aplica transformaciones de limpieza y filtrado a un DataFrame de preguntas.

    Descripción
    -----------
    - Filtra las etiquetas de cada fila usando `most_common`; si no quedan etiquetas, asigna None.
    - Extrae y limpia el texto HTML de la columna 'Body' usando BeautifulSoup y `clean_text`,
      además de eliminar cualquier etiqueta HTML restante.
    - Asegura que la columna 'Title' sea de tipo cadena.

    Parámetros
    ----------
    data: pandas.DataFrame
        DataFrame que debe contener las columnas:
        - 'Tag': listas de etiquetas por fila.
        - 'Body': texto con posible HTML.
        - 'Title': valores a convertir a texto.

    Retorna
    -------
    pandas.DataFrame
        Mismo DataFrame con las transformaciones aplicadas:
        - 'Tag': lista de etiquetas comunes o None.
        - 'Body': texto limpio sin HTML, en minúsculas y con contracciones expandidas.
        - 'Title': valores convertidos a str.
    """
    data['Tag'] = data['Tag'].apply(lambda x: most_common(x))
    data['Tag'] = data['Tag'].apply(lambda x: x if len(x) > 0 else None)
    data = data.dropna(subset=['Tag'])

    data['Body'] = data['Body'].apply(lambda x: BeautifulSoup(x).get_text())
    data['Body'] = data['Body'].apply(lambda x: clean_text(x))
    data['Body']= data['Body'].apply(lambda x: re.sub('<[^<]+?>','',x))

    data['Title'] = data['Title'].astype(str)

    return data

In [12]:
questions = read_data('../data/input/Questions.csv')

In [13]:
tags = read_data('../data/input/Tags.csv')
tags = tags.dropna(subset=['Tag'])

In [14]:
questions = drop_columns(questions, ['OwnerUserId', 'CreationDate', 'ClosedDate','Score'])

In [15]:
grouped_tags = group_tags(tags, "Id", "Tag")
grouped_tags.head(5)

,Id,Tag
0,80,"[flex, actionscript-3, air]"
1,90,"[svn, tortoisesvn, branch, branching-and-merging]"
2,120,"[sql, asp.net, sitemap]"
3,180,"[algorithm, language-agnostic, colors, color-s..."
4,260,"[c#, .net, scripting, compiler-construction]"


In [16]:
data_merged = merge_datasets(questions,grouped_tags,'Id')
data_merged.head(10)

,Id,Title,Body,Tag
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]"
1,90,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch, branching-and-merging]"
2,120,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net, sitemap]"
3,180,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,"[algorithm, language-agnostic, colors, color-s..."
4,260,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net, scripting, compiler-construction]"
5,330,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,"[c++, oop, class, nested-class]"
6,470,Homegrown consumption of web services,<p>I've been writing a few web services for a ...,"[.net, web-services]"
7,580,Deploying SQL Server Databases from Test to Live,<p>I wonder how you guys manage deployment of ...,"[sql-server, sql-server-2005, deployment, rele..."
8,650,Automatically update version number,<p>I would like the version property of my app...,"[c#, visual-studio, versioning]"
9,810,Visual Studio Setup Project - Per User Registr...,<p>I'm trying to maintain a Setup Project in <...,"[windows, visual-studio, registry, installation]"


In [17]:
tags_features = get_frecuency_tags(data_merged, 'Tag')
tags_features

['javascript',
 'java',
 'c#',
 'php',
 'android',
 'jquery',
 'python',
 'html',
 'c++',
 'ios',
 'mysql',
 'css',
 'sql',
 'asp.net',
 'objective-c',
 'ruby-on-rails',
 '.net',
 'c',
 'iphone',
 'angularjs',
 'arrays',
 'sql-server',
 'json',
 'ruby',
 'r',
 'ajax',
 'regex',
 'xml',
 'node.js',
 'asp.net-mvc',
 'linux',
 'django',
 'wpf',
 'database',
 'swift',
 'xcode',
 'string',
 'excel',
 'vb.net',
 'windows',
 'spring',
 'wordpress',
 'eclipse',
 'html5',
 'multithreading',
 'oracle',
 'git',
 'facebook',
 'forms',
 'bash',
 'image',
 'osx',
 'twitter-bootstrap',
 'mongodb',
 'winforms',
 'vba',
 'algorithm',
 'apache',
 'performance',
 'matlab',
 'swing',
 'visual-studio',
 'entity-framework',
 'linq',
 'postgresql',
 'hibernate',
 'list',
 'python-2.7',
 'css3',
 'scala',
 'ruby-on-rails-3',
 'qt',
 '.htaccess',
 'web-services',
 'function',
 'sqlite',
 'perl',
 'excel-vba',
 'shell',
 'rest',
 'api',
 'sql-server-2008',
 'uitableview',
 'laravel',
 'codeigniter',
 'file',
 '

In [18]:
data_merged = data_transformation(data_merged)

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_12176\2101070157.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Body'] = data['Body'].apply(lambda x: BeautifulSoup(x).get_text())
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_12176\2101070157.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Body'] = data['Body'].apply(lambda x: clean_text(x))
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_12176\2101070157.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [20]:
data_merged

,Id,Title,Body,Tag
2,120,ASP.NET Site Maps,has anyone got experience creating sql-based a...,"[sql, asp.net]"
3,180,Function for creating color wheels,this is something i have pseudo-solved many t...,[algorithm]
4,260,Adding scripting functionality to .NET applica...,i have a little game written in c#. it uses a ...,"[c#, .net]"
5,330,Should I use nested classes in this case?,i am working on a collection of classes used f...,"[c++, class]"
6,470,Homegrown consumption of web services,i have been writing a few web services for a ...,"[.net, web-services]"
...,...,...,...,...
1264207,40143170,How to bind a model to resource store method i...,i am trying to assign a product model for stor...,[laravel]
1264208,40143190,How to execute multiline python code from a ba...,i need to extend a shell script (bash). as i a...,"[python, bash]"
1264209,40143210,URL routing in PHP (MVC),i am building a custom mvc project and i have ...,"[php, .htaccess]"
1264211,40143340,Obfuscating code in android studio,under minifyenabled i changed from false to tr...,[android]


In [19]:
data_merged.to_parquet('../data/output/StackOverflow.parquet',index=False)